# SuSiE.ash Simulation Scripts

## Table of Contents

- [Goal & Setup](#goal)
- [LD Block Extraction](#ld-blocks-extraction-for-both-sparse-and-oligogenic-settings)
- [Helper Functions for Main Simulation](#helper-functions-for-main-simulation-scripts)
- [Sparse Simulation](#sparse-simulation-script)

## Goal

We aim to show the performance SuSiE, SuSiE.ash (Marginal & Random Effects), SuSiE-inf, and Fineboost in various realistic expression quantitative trait loci (eQTL) sparse and oligogenic simulation settings.

## Sparse Simulation Setup

The sparse simulations use data from UK Biobank where we randomly sampled 150 LD blocks, for each simulation setting, across chromosomes 1-22 and derive the associated genotype matrices to serve as the basis for our simulations. For the first simulation setting, we take the first 1000 variants and 1500 individuals, and for the second simulation setting, we take between 3000 to 12000 variants and 1500 individauls. For the first simulation setting, we run all pairwise combinations of the number of effect variables = {1, 2, 3, 4, 5} and heritability = {0.05, 0.10, 0.20, 0.40} twice for each genotype matrix, resulting in a total of 2 x 150 x 5 x 4 = 6000 data sets for the first simulation setting. For the second simulation setting, we simply set the number of effect variables = 10 and the heritability = 0.30 and run this scenario twice for each genotype matrix, resulting in a total of 2 x 150 = 300 data sets. All LD blocks have a missing rate < 0.05, minor allele frequency (MAF) > 0.05, filter out all variants with zero (or near-zero variance), and utilize mean imputation for any missing data. Finally, we will repeat the first and second simulation setting for three types of LD structures between causal variants: i) no LD among causal variants (|r| < 0.05), ii) minimal LD (|r| < 0.30) among causal variants, and iii) randomly assigned LD between causal variants.

## Oligogenic Simulation Setup

Similar to the sparse simulations, the oligogenic simulations also use data from UK Biobank. We randomly sampled 150 LD blocks across chromosomes 1-22 and derive the associated genotype matrices to serve as the basis for our simulations. We will take between 5000 and 8000 variants and 1500 individuals, and will run three separate levels of heritability = {0.15, 0.30, 0.50}. For each level, we will utilize each genotype matrix twice, resulting in a total of 3 x 2 x 150 = 900 data sets. We applied the same QC measures as the sparse simulations, however, we will only randomly assign LD between causal variants.

## LD Blocks Extraction (for both Sparse and Oligogenic Settings)

### Filter LD Blocks

LD blocks have already passed preliminary QC filters, such as filtering out variants with low MAF (< 0.05) and high missing rate (> 0.10). However, we will apply additional QC measures and refine the genotype matrices. 

In [ ]:
# note you will need to set the proper Access Key ID and Secret Access Key ID to the S3 bucket.
Sys.setenv("AWS_ACCESS_KEY_ID" = "KEY",
           "AWS_SECRET_ACCESS_KEY" = "SECRET KEY",
           "AWS_DEFAULT_REGION" = "us-east-1")

library(aws.s3)
library(stringr)
library(tidyverse)

output_folder <- "ftp_fgc_xqtl/interactive_sessions/apm2217/susie-ash-data/UKBB_data"
output_files <- get_bucket(bucket = "statfungen", prefix = output_folder, max = Inf)

# filter by log files
log_files <- sapply(output_files, function(x) x$Key)
log_files <- log_files[grep("\\.log$", log_files)]

# initialize df
results_df <- data.frame(
  LD_Block = character(),
  Variants_Passed_QC = integer(),
  Genotyping_Rate = numeric(),
  stringsAsFactors = FALSE
)

# read each .log file
for (log_file in log_files) {
  # Read the .log file directly from S3 into R
  log_content <- s3read_using(
    FUN = readLines,
    object = log_file,
    bucket = "statfungen"
  )
  
  # extract the number of variants that passed QC
  variants_line <- grep("variants and .* people pass filters and QC", log_content, value = TRUE)
  variants_passed <- as.integer(str_extract(variants_line, "\\d+"))
  
  # extract the genotyping rate
  geno_rate_line <- grep("Total genotyping rate is", log_content, value = TRUE)
  genotyping_rate <- as.numeric(str_extract(geno_rate_line, "\\d+\\.\\d+"))
  
  # get the LD block identifier from the file name
  ld_block_id <- basename(log_file)
  ld_block_id <- sub("\\.log$", "", ld_block_id)
  
  # append the extracted data to the results data frame
  results_df <- rbind(
    results_df,
    data.frame(
      LD_Block = ld_block_id,
      Variants_Passed_QC = variants_passed,
      Genotyping_Rate = genotyping_rate,
      stringsAsFactors = FALSE
    )
  )
}

# filter for at least p variants and missing rate < 0.05
filtered_ld_blocks <- subset(
  results_df,
  Variants_Passed_QC >= 1000 &   # sparse setting 1
  # Variants_Passed_QC >= 3000 & # sparse setting 2
  # Variants_Passed_QC >= 5000 & # oligogenic setting
    Genotyping_Rate >= 0.95
)

### Randomly Sample 150 LD Blocks from List and Generate Script to Run Genotype Extraction

In [ ]:
set.seed(6868)   # Sparse Simulation 1 Seed
# set.seed(8686) # Sparse Simulation 2 Seed
# set.seed(6688) # Oligogenic Simulation Seed
sampled_blocks <- sample(filtered_ld_blocks$LD_Block, size = 150, replace = FALSE)

chromosome_ids <- str_extract(sampled_blocks, "(?<=chr)[^_]+")

chromosome_counts <- table(chromosome_ids)
chromosome_df <- as.data.frame(chromosome_counts)
colnames(chromosome_df) <- c("Chromosome", "Count")


# function to map chromosome identifiers to numeric values
map_chr_to_num <- function(chr) {
  if (chr %in% as.character(1:22)) {
    return(as.numeric(chr))
  } else {
    return(NA)
  }
}

# apply the function to create a numeric representation
chromosome_df$Chromosome_numeric <- sapply(chromosome_df$Chromosome, map_chr_to_num)
chromosome_df <- chromosome_df[order(chromosome_df$Chromosome_numeric), ]
chromosome_df$Chromosome <- factor(chromosome_df$Chromosome, levels = chromosome_df$Chromosome)

# create the bar plot of distribution chromosome counts
ggplot(chromosome_df, aes(x = factor(Chromosome, levels = c(seq(1,22))), y = Count)) +
  geom_bar(stat = "identity", fill = "steelblue") +
  xlab("Chromosome") +
  ylab("Number of LD Blocks Sampled") +
  ggtitle("Distribution of Sampled LD Blocks Across Chromosomes") +
  theme_minimal()

# create the commands_to_submit.txt file
commands_file <- "commands_to_submit.txt"
file_conn <- file(commands_file, open = "w")

# iterate over each LD block and write commands to the file
for (ld_block_name in sampled_blocks) {
  # create the command
  command <- paste0("Rscript /home/apm2217/data/process_ld_block.R ", ld_block_name)

  # write the command to the file
  writeLines(command, file_conn)
}

# close the file connection
close(file_conn)

cat("Commands file 'commands_to_submit.txt' created successfully.\n")


### Script to Extract Genotype Matrices

Below is the "process_ld_block.R" script that our "commands_to_submit.txt" file references. We will submit each of these commands to the cloud for processing.

In [ ]:
library(bigsnpr)

# get the LD block name from command-line arguments
args <- commandArgs(trailingOnly = TRUE)
if (length(args) == 0) {
  stop("No LD block name provided. Usage: Rscript process_ld_block.R ld_block_name")
}
ld_block_name <- args[1]

# define your data and output paths
data_path <- "/home/apm2217/data/" # this points to the S3 bucket containing the UKBB data
output_path <- "/home/apm2217/output/" 

# function to process a single LD block
process_ld_block <- function(ld_block_name) {
  message("Processing LD block: ", ld_block_name)

  # define file paths
  bedfile <- file.path(data_path, paste0(ld_block_name, ".bed"))
  bimfile <- file.path(data_path, paste0(ld_block_name, ".bim"))
  famfile <- file.path(data_path, paste0(ld_block_name, ".fam"))

  # check if files exist
  if (!file.exists(bedfile) || !file.exists(bimfile) || !file.exists(famfile)) {
    stop("One or more PLINK files for ", ld_block_name, " are missing.")
  }

  # create a temporary directory for processing
  temp_dir <- tempfile(pattern = "temp_ld_block_")
  dir.create(temp_dir)

  # Copy PLINK files to temporary directory
  file.copy(bedfile, temp_dir)
  file.copy(bimfile, temp_dir)
  file.copy(famfile, temp_dir)

  # define paths to copied files
  temp_bedfile <- file.path(temp_dir, basename(bedfile))
  temp_bimfile <- file.path(temp_dir, basename(bimfile))
  temp_famfile <- file.path(temp_dir, basename(famfile))

  # read PLINK files using bigsnpr
  backingfile <- file.path(temp_dir, paste0(ld_block_name, "_bk"))

  # convert PLINK files to bigSNP format (creates .rds and .bk files)
  snp_readBed(temp_bedfile, backingfile = backingfile)

  # attach the bigSNP object
  obj.bigSNP <- snp_attach(paste0(backingfile, ".rds"))
  G <- obj.bigSNP$genotypes
  sample_ids <- obj.bigSNP$fam$sample.ID
  variant_ids <- obj.bigSNP$map$marker.ID

  # check dimensions
  num_individuals <- nrow(G)
  num_variants <- ncol(G)

  message("Number of individuals: ", num_individuals)
  message("Number of variants: ", num_variants)

  # extract the first n individuals
  num_individuals_to_sample <- 1500 # 
  sampled_individuals <- 1:num_individuals_to_sample

  # extract the first p variants

  ## Sparse Setting 1
  num_variants_to_sample <- 1000 

  ## Sparse Setting 2
  # set.seed(8686)
  # num_variants_to_sample <- round(runif(1, 3000, min(num_variants,12000)))

  ## Oligogenic Setting 
  # set.seed(6688)
  # num_variants_to_sample <- round(runif(1, 5000, min(num_variants,8000)))
  sampled_variants <- 1:num_variants_to_sample

  # subset the genotype matrix
  G_sub <- G[sampled_individuals, sampled_variants]

  # Create a list to save
  genotype_data <- list(
    genotypes = G_sub,
    sample_ids = sample_ids[sampled_individuals],
    variant_ids = variant_ids[sampled_variants],
    chromosome = obj.bigSNP$map$chromosome[sampled_variants],
    physical_pos = obj.bigSNP$map$physical.pos[sampled_variants],
    alleles = obj.bigSNP$map[, c("allele1", "allele2")][sampled_variants, ]
  )

  # save the genotype data to an RDS file in the output directory
  output_file <- file.path(output_path, paste0(ld_block_name, "_matrix.rds"))
  saveRDS(genotype_data, file = output_file)

  message("Saved processed data to ", output_file)

  # clean up temporary files
  unlink(temp_dir, recursive = TRUE)

  message("Completed processing for ", ld_block_name)
}

# call the function with the provided LD block name
process_ld_block(ld_block_name)

### Shell Script to Submit Cloud Job

In [ ]:
#!/bin/bash
username=apm2217
./src/mm_batch.sh \
  --job-script ./commands_to_submit.txt \
  -c 4 -m 64 \
  --job-size 150 \
  --parallel-commands 4 \
  --mount statfungen/ftp_fgc_xqtl/interactive_sessions/$username/susie-ash-data/UKBB_data:/home/$username/data \
  --mount statfungen/ftp_fgc_xqtl/interactive_sessions/$username/sparse_LD_blocks_scenario_1:/home/$username/output \
  --mountOpt "mode=r" "mode=rw" \
  --cwd "/home/$username/data" \
  --imageVolSize 20 \
  --opcenter 3.82.198.55 \
  --no-fail-fast

### Script for QC & Precomputing Values for SuSiE.ash and SuSiE-inf

For quality control, in addition to missing rate and MAF filters from earlier, we also use mean imputation for any missing data and remove variants with zero (or near-zero) variance. Additionally, SuSiE.ash and SuSiE-inf both can be drastically sped up if we use precomputed matrices for the LD matrix, eigen values/vectors, and more. 

In [ ]:
library(Matrix)
input_dir <- "/home/apm2217/data"   
output_dir <- "/home/apm2217/output"

args <- commandArgs(trailingOnly = TRUE)
ld_block_names <- args

# ensure output directory exists
if (!dir.exists(output_dir)) {
  dir.create(output_dir, recursive = TRUE)
}

# function to mean-impute missing values
mean_impute <- function(geno) {
  # Compute column-wise means excluding NAs
  col_means <- apply(geno, 2, function(x) mean(x, na.rm = TRUE))

  # replace NAs with the corresponding column mean
  for (i in seq_along(col_means)) {
    na_indices <- which(is.na(geno[, i]))
    if (length(na_indices) > 0) {
      geno[na_indices, i] <- col_means[i]
    }
  }

  return(geno)
}

# function to process a single LD block
process_ld_block <- function(ld_block_name) {
  # construct the full path to the LD block file
  input_file <- file.path(input_dir, paste0(ld_block_name, "_matrix.rds"))

  # check if the input file exists
  if (!file.exists(input_file)) {
    cat("Input file does not exist:", input_file, "\n")
    return(NULL)
  }

  cat("Processing LD block file:", input_file, "\n")

  # load the LD block from the local file system
  ld_block_obj <- readRDS(input_file)

  # check if 'genotypes' exist in the LD block object
  if (!"genotypes" %in% names(ld_block_obj)) {
    cat("The LD block object does not contain 'genotypes'. Skipping.\n")
    return(NULL)
  }

  # extract and mean-impute the genotype matrix
  X <- mean_impute(ld_block_obj$genotypes)

  # remove columns with zero or near-zero variance
  sd_X <- apply(X, 2, sd)
  threshold <- 1e-8  # Adjust as needed
  columns_to_remove <- which(sd_X < threshold)

  if (length(columns_to_remove) > 0) {
    cat("Removing", length(columns_to_remove), "columns with zero or near-zero variance.\n")
    X <- X[, -columns_to_remove, drop = FALSE]
  }

  if (ncol(X) == 0) {
    cat("No columns left after removing zero or near-zero variance columns. Skipping.\n")
    return(NULL)
  }

  # scale the genotype matrix
  X_scaled <- scale(X)

  # compute XtX
  cat("Computing XtX\n")
  XtX <- crossprod(X_scaled)

  # compute LD matrix
  n_samples <- nrow(X)
  LD <- XtX / n_samples

  # compute eigenvalues and eigenvectors
  cat("Computing eigenvalues of LD matrix\n")
  eig <- eigen(LD, symmetric = TRUE)
  V <- eig$vectors
  Dsq <- pmax(n_samples * eig$values, 0)

  # compute VtXt
  VtXt <- t(V) %*% t(X_scaled)

  # prepare the result list
  result <- list(
    X = X,        # unscaled processed genotype matrix
    XtX = XtX,
    LD = LD,
    V = V,
    Dsq = Dsq,
    VtXt = VtXt
  )

  # Construct the output file path
  output_file <- file.path(output_dir, paste0(ld_block_name, "_processed.rds"))
  cat("Saving processed matrices to:", output_file, "\n")

  # Save the result to the output directory
  saveRDS(result, file = output_file)
}

# Process each LD block
for (ld_block_name in ld_block_names) {
  process_ld_block(ld_block_name)
}

cat("All LD blocks have been processed.\n")


### Generating Commands Script

This requires the same `sampled_blocks` object from earlier, a vector containing the strings of the LD blocks sampled. This can be submitted to cloud using same shell script from above just with adjusted input and output folder pathways.

In [ ]:
# path to the processing script
processing_script <- "/home/apm2217/data/LD_blocks_precomputations.R"

# create the commands_to_submit.txt file
commands_file <- "commands_to_submit.txt"
file_conn <- file(commands_file, open = "w")

# iterate over each LD block and write commands to the file
for (ld_block_name in sampled_blocks) {
  # Create the command to process this LD block
  command <- paste(
    "Rscript", processing_script, ld_block_name
  )

  # write the command to the file
  writeLines(command, file_conn)
}

# close the file connection
close(file_conn)

cat("Commands file 'commands_to_submit.txt' created successfully.\n")

## Helper Functions for Main Simulation Scripts

### Phenotype Generation

#### Sparse Simulation

Below is the function used to generate our sparse data expression levels. The user must enter in a i) genotype matrix, ii) number of effect variables, iii) desired heritability level, iv) ld_mode (this refers to the LD between causal variants and can either be `random`, `minimal` (|r| < 0.05), or `low` |r| < 0.30). Optionally, the user can input their own seed and LD matrix (to save on computation time).

In [ ]:
# ================================
# sparse_data_generation.R
# ================================

library(simxQTL)

# helper function to sample causal indices that satisfy an LD threshold
get_valid_causal <- function(G, ncausal, ld_threshold, max_attempts = 10000, ld_matrix = NULL) {
  snp_indices <- seq_len(ncol(G))
  for (attempt in 1:max_attempts) {
    causal_indices <- sort(sample(snp_indices, ncausal))
    if (!is.null(ld_matrix)) {
      # use the provided LD matrix (assumed to already be absolute)
      corr_mat <- ld_matrix[causal_indices, causal_indices]
    } else {
      # compute correlation matrix and take absolute values
      corr_mat <- abs(cor(G[, causal_indices]))
    }
    # set diagonal and lower triangle to zero (only off-diagonals matter)
    corr_mat[lower.tri(corr_mat, diag = TRUE)] <- 0
    if (max(corr_mat) < ld_threshold) {
      return(causal_indices)
    }
  }
  stop("Could not find a set of causal variants with LD (|r|) below ", ld_threshold, " after ", max_attempts, " attempts.")
}

generate_sparse_eqtl_data <- function(X, K = 10, h2 = 0.3, seed = NULL,
                                      ld_mode = "random", ld_matrix = NULL, max_attempts = 10000) {
  if (!is.null(seed)) set.seed(seed)

  n_samples <- nrow(X)
  n_features <- ncol(X)

  # parse K to get the number of causal SNPs
  causal_info <- parse_num_causal_snps(as.character(K))
  if (causal_info$is_pct) {
    n_causal <- max(1, round(causal_info$value * n_features))
  } else {
    n_causal <- min(causal_info$value, n_features)
  }

  # determine causal indices based on ld_mode
  if (ld_mode == "random") {
    causal_indices <- sort(sample(seq_len(n_features), n_causal, replace = FALSE))
  } else if (ld_mode == "minimal_ld") {
    causal_indices <- get_valid_causal(G = X, ncausal = n_causal,
                                       ld_threshold = 0.05, max_attempts = max_attempts,
                                       ld_matrix = ld_matrix)
  } else if (ld_mode == "low_ld") {
    causal_indices <- get_valid_causal(G = X, ncausal = n_causal,
                                       ld_threshold = 0.30, max_attempts = max_attempts,
                                       ld_matrix = ld_matrix)
  } else {
    stop("Invalid ld_mode. Choose from 'random', 'minimal_ld', or 'low_ld'.")
  }

  # create beta vector and assign effect sizes from N(0, 0.6^2) for the causal SNPs
  beta <- rep(0, n_features)
  beta[causal_indices] <- rnorm(length(causal_indices), mean = 0, sd = 0.6)

  # compute the latent genetic effect
  g <- as.vector(X %*% beta)

  # generate phenotype using the GitHub simulate_polygenic_trait function
  y <- simulate_polygenic_trait(g, h2)

  # compute estimated heritability
  var_g <- var(g)
  var_e <- var(y - g)
  h2_estimated <- var_g / (var_g + var_e)

  return(list(
    X = X,
    y = as.vector(y),
    beta = beta,
    causal_indices = causal_indices,
    var_epsilon = var_e,
    h2_input = h2,
    h2_estimated = h2_estimated
  ))
}

#### Oligogenic Simulation

Note that the `generate_eqtl_data()` function is included in the simxQTL repo, however, it is included here for completeness. We also included the `is_causal()` function which assigns variants that meet a user-defined PVE threshold to be causal.

In [ ]:
# ================================
# oligogenic_data_generation.R
# ================================

library(simxQTL)

generate_eqtl_data <- function(G,
                               h2_total = 0.3,
                               prop_h2_sparse = 0.65,
                               prop_h2_oligogenic = 0.20,
                               prop_h2_infinitesimal = 0.15,
                               prop_h2_sentinel = 0.7,
                               n_oligogenic = 20,
                               mixture_props = c(0.75, 0.25),
                               mixture_sds = c(0.0025, 0.005),
                               n_other_sparse = 2,
                               standardize = TRUE,
                               seed = NULL) {
  # Minimal input validation for proportions
  if (abs(prop_h2_sparse + prop_h2_oligogenic + prop_h2_infinitesimal - 1) > 1e-6) {
    stop("The sum of prop_h2_sparse, prop_h2_oligogenic, and prop_h2_infinitesimal must equal 1.")
  }
  if (abs(sum(mixture_props) - 1) > 1e-6) {
    stop("mixture_props must sum to 1.")
  }
  
  if (!is.null(seed)) set.seed(seed)
  
  # Standardize genotype matrix if requested
  if (standardize) {
    G <- scale(G)
  }
  
  n_samples <- nrow(G)
  n_features <- ncol(G)
  
  # Allocate heritability components
  h2_sparse <- h2_total * prop_h2_sparse
  h2_oligogenic <- h2_total * prop_h2_oligogenic
  h2_infinitesimal <- h2_total * prop_h2_infinitesimal
  
  # 1. Sparse Effects (sentinel + additional sparse SNPs)
  sparse_res <- simulate_sparse_effects(G, h2_sparse, prop_h2_sentinel, n_other_sparse)
  beta_sparse <- sparse_res$beta
  sentinel_index <- sparse_res$sentinel_index
  other_sparse_indices <- sparse_res$other_sparse_indices
  sparse_indices <- c(sentinel_index, other_sparse_indices)
  
  # 2. Oligogenic Effects
  non_sparse_indices <- setdiff(1:n_features, sparse_indices)
  oligo_res <- simulate_oligogenic_effects(G, h2_oligogenic, n_oligogenic, mixture_props, mixture_sds, non_sparse_indices)
  beta_oligo <- oligo_res$beta
  oligogenic_indices <- oligo_res$oligogenic_indices
  
  # 3. Infinitesimal Effects (remaining SNPs)
  infinitesimal_indices <- setdiff(non_sparse_indices, oligogenic_indices)
  beta_inf <- simulate_infinitesimal_effects(G, h2_infinitesimal, infinitesimal_indices)
  
  # Combine all effect components
  beta <- beta_sparse + beta_oligo + beta_inf
  
  # Generate latent genetic component and phenotype
  g <- as.vector(G %*% beta)
  var_g <- var(g)
  var_epsilon <- var_g * (1 - h2_total) / h2_total
  epsilon <- rnorm(n_samples, 0, sqrt(var_epsilon))
  y <- g + epsilon
  
  # Calculate realized heritability components
  var_y <- var(y)
  h2_sentinel_actual <- var(G[, sentinel_index] * beta[sentinel_index]) / var_y
  h2_sparse_actual <- var(as.vector(G[, sparse_indices] %*% beta[sparse_indices])) / var_y
  h2_oligogenic_actual <- var(as.vector(G[, oligogenic_indices] %*% beta[oligogenic_indices])) / var_y
  h2_infinitesimal_actual <- var(as.vector(G[, infinitesimal_indices] %*% beta_inf[infinitesimal_indices])) / var_y
  h2_total_actual <- var(as.vector(G %*% beta)) / var_y
  
  return(list(
    G = G,                # Standardized genotype matrix
    y = y,                # Simulated phenotype (on its generated scale)
    beta = beta,
    h2_total = h2_total_actual,
    h2_sparse = h2_sparse_actual,
    h2_sentinel = h2_sentinel_actual,
    h2_oligogenic = h2_oligogenic_actual,
    h2_infinitesimal = h2_infinitesimal_actual,
    sentinel_index = sentinel_index,
    other_sparse_indices = other_sparse_indices,
    oligogenic_indices = oligogenic_indices,
    infinitesimal_indices = infinitesimal_indices,
    residual_variance = var_epsilon
  ))
}

# function to identify causal SNPs based on PVE threshold
is_causal <- function(beta, residual_variance, pve_threshold) {

  # compute variance explained by each SNP (since Var(X_j) = 1), assumes standardized genotype matrix
  variance_explained <- beta^2

  # compute total genetic variance
  var_g <- sum(variance_explained)

  # compute total variance (genetic variance + residual variance)
  total_variance <- var_g + residual_variance

  # compute PVE for each SNP
  proportion_var_explained <- variance_explained / total_variance

  # define causal SNPs based on the current PVE threshold
  causal_SNPs <- which(proportion_var_explained > pve_threshold)
  return(causal = causal_SNPs)
}

### Wrapper Function to Run Methods

In [ ]:
# ================================
# run_methods.R
# ================================
# This file contains wrapper functions for running different methods:
# - run_susie(): Standard SuSiE method.
# - run_susie_ash(): SuSiE.ash (Marginal) using susie_ash_RE_Marg().
# - run_susie_inf(): SuSiE-inf method.
# - run_fineboost(): Fineboost using colocboost().


# Wrapper for SuSiE
run_susie <- function(data, L, intercept = TRUE, standardize = TRUE) {
  cat("Starting SuSiE\n")
  out <- susie(
    X = data$X,
    y = data$y,
    L = L,
    intercept = intercept,
    standardize = standardize
  )
  return(out)
}

# Wrapper for SuSiE.ash (Marginal)
run_susie_ash <- function(data, precomp, L, K.length, upper_bound, intercept = TRUE, standardize = TRUE) {
  cat("Starting SuSiE.ash (Marginal)\n")

  # Manually scale X and y if intercept or standardize is TRUE.
  X_in <- if (intercept || standardize) scale(data$X, center = intercept, scale = standardize) else data$X
  y_in <- if (intercept || standardize) scale(data$y, center = intercept, scale = standardize) else data$y

  out <- susie_ash_RE_Marg(
    X                = X_in,
    y                = y_in,
    L                = L,
    verbose          = FALSE,
    coverage         = 0.95,
    XtX              = precomp$XtX,
    LD               = precomp$LD,
    V                = precomp$V,
    Dsq              = precomp$Dsq,
    VtXt             = precomp$VtXt,
    update_ash_sigma = FALSE,
    K.length         = K.length,
    upper_bound      = upper_bound
  )
  return(out)
}

# Wrapper for SuSiE-inf
run_susie_inf <- function(data, precomp, L, intercept = TRUE, standardize = TRUE) {
  cat("Starting SuSiE-inf\n")

  # Manually scale X and y if needed.
  X_in <- if (intercept || standardize) scale(data$X, center = intercept, scale = standardize) else data$X
  y_in <- if (intercept || standardize) scale(data$y, center = intercept, scale = standardize) else data$y

  out <- susie_inf(
    X       = X_in,
    y       = y_in,
    L       = L,
    verbose = FALSE,
    coverage = 0.95,
    XtX     = precomp$XtX,
    LD      = precomp$LD,
    V       = precomp$V,
    Dsq     = precomp$Dsq
  )
  return(out)
}

# Wrapper for Fineboost
run_fineboost <- function(data, null_max, intercept = TRUE, standardize = TRUE) {
  cat("Starting Fineboost\n")
  out <- colocboost(
    X              = data$X,
    Y              = data$y,
    check_null_max = null_max,
    intercept      = intercept,
    standardize    = standardize
  )
  return(out)
}


### Performance Evalution Function

In [ ]:
# ================================
# evaluate_method_performance.R
# ================================
# This file defines a unified calc_metrics() function to compute:
# - Average Credible Set (CS) Size
# - Coverage (proportion of CSs capturing a causal variant)
# - CS-based FDR and Recall
#
# The function adapts to different model outputs based on the provided method.

calc_metrics <- function(mod, method, X, causal) {
  # Define test.cs based on the method
  if (method == "susie") {
    test.cs <- susie_get_cs(mod, X = X, coverage = 0.95)$cs
  } else if (method == "fineboost") {
    test.cs <- mod$ucos_details$ucos$ucos_index
  } else if (method %in% c("susie_ash", "susie_inf")) {
    test.cs <- mod$sets
  } else {
    stop("Unknown method specified for metric calculation.")
  }

  # Initialize metrics
  cs_size   <- 0
  coverage  <- 0
  cs_fdr    <- 0
  cs_recall <- 0

  if (length(test.cs) > 0) {
    cs_size  <- length(unlist(test.cs)) / length(test.cs)
    coverage <- sum(sapply(test.cs, function(cs) any(causal %in% cs))) / length(test.cs)

    TP_fdr <- sum(sapply(test.cs, function(cs) any(cs %in% causal)))
    FP_fdr <- length(test.cs) - TP_fdr
    cs_fdr <- if ((TP_fdr + FP_fdr) > 0) FP_fdr / (TP_fdr + FP_fdr) else NA

    TP_recall <- sum(causal %in% unlist(test.cs))
    FN_recall <- length(causal) - TP_recall
    cs_recall <- TP_recall / (TP_recall + FN_recall)
  }

  return(list(
    cs_size   = cs_size,
    coverage  = coverage,
    cs_fdr    = cs_fdr,
    cs_recall = cs_recall
  ))
}

# Main function to aggregate metrics from all methods
evaluate_method_performance <- function(susie_out, susie_ash_out, susie_inf_out, fineboost_out, causal, data, precomp = NULL) {
  # data$X is used directly.
  susie_metrics      <- calc_metrics(susie_out,      method = "susie",      X = data$X, causal = causal)
  susie_ash_metrics  <- calc_metrics(susie_ash_out,  method = "susie_ash",  X = data$X, causal = causal)
  susie_inf_metrics  <- calc_metrics(susie_inf_out,  method = "susie_inf",  X = data$X, causal = causal)
  fineboost_metrics  <- calc_metrics(fineboost_out,  method = "fineboost",  X = data$X, causal = causal)

  # Create a summary metrics table
  metrics_table <- data.frame(
    Model     = c("SuSiE", "SuSiE.ash (Marginal)", "SuSiE-inf", "Fineboost"),
    CS_Size   = c(susie_metrics$cs_size,
                  susie_ash_metrics$cs_size,
                  susie_inf_metrics$cs_size,
                  fineboost_metrics$cs_size),
    Coverage  = c(susie_metrics$coverage,
                  susie_ash_metrics$coverage,
                  susie_inf_metrics$coverage,
                  fineboost_metrics$coverage),
    CS_FDR    = c(susie_metrics$cs_fdr,
                  susie_ash_metrics$cs_fdr,
                  susie_inf_metrics$cs_fdr,
                  fineboost_metrics$cs_fdr),
    CS_Recall = c(susie_metrics$cs_recall,
                  susie_ash_metrics$cs_recall,
                  susie_inf_metrics$cs_recall,
                  fineboost_metrics$cs_recall)
  )

  return(list(metrics = metrics_table))
}

## Sparse Simulation Script

In [ ]:
# ================================
# Sparse Simulation Script
# ================================

# Load required libraries
library(susieR)
library(dplyr)
library(magrittr)

# Load Fineboost source files
for (file in list.files("fineboost", pattern = "\\.R$", full.names = TRUE)) {
  source(file)
}

# Source helper functions from the helper_functions folder
source("helper_functions/sparse_data_generation.R")
source("helper_functions/run_methods.R")
source("helper_functions/evaluate_method_performance.R")

##### STILL NEED simxQTL and susieash #####

# Define the simulation function
simulation <- function(num_simulations = NULL,
                       h2_total        = NULL,
                       K               = NULL,
                       L               = NULL,
                       null_max        = NULL,
                       ld_mode         = NULL,
                       K.length        = NULL,
                       upper_bound     = NULL,
                       LD_blocks_dir   = "LD_blocks_precomputed") {

  # Set Default Values if parameters are missing
  if (is.null(num_simulations)) num_simulations <- 2
  if (is.null(h2_total))        h2_total        <- 0.3
  if (is.null(K))               K               <- 5
  if (is.null(L))               L               <- 10
  if (is.null(null_max))        null_max        <- 0.02
  if (is.null(ld_mode))         ld_mode         <- "random"
  if (is.null(K.length))        K.length        <- 20
  if (is.null(upper_bound))     upper_bound     <- 2

  # Parse Command-Line Arguments (if any)
  for (arg in commandArgs(trailingOnly = TRUE)) {
    eval(parse(text = arg))
  }

  # List available LD block files and check count
  ld_block_files <- list.files(path = LD_blocks_dir, pattern = "\\.rds$", full.names = TRUE)
  if (length(ld_block_files) < num_simulations) {
    stop("Not enough LD block files.")
  }
  ld_block_files <- ld_block_files[1:num_simulations]

  # Container for simulation metrics
  all_metrics <- vector("list", num_simulations)

  # Loop over simulation replicates
  for (i in seq_len(num_simulations)) {
    cat("Running simulation", i, "of", num_simulations, "\n")

    # Load precomputed LD block data
    ld_block <- readRDS(ld_block_files[i])

    # Store precomputed matrices (these can be large)
    precomp <- list(
      XtX  = ld_block$XtX,
      LD   = ld_block$LD,
      V    = ld_block$V,
      Dsq  = ld_block$Dsq,
      VtXt = ld_block$VtXt
    )

    # Generate simulation data with seed equal to the replicate index
    data <- generate_sparse_eqtl_data(
      X         = ld_block$X,
      K         = K,
      h2        = h2_total,
      ld_mode   = ld_mode,
      ld_matrix = ld_block$LD,
      seed      = i
    )

    # Run methods via the wrapper functions
    susie_out      <- run_susie(data, L, intercept = TRUE, standardize = FALSE)
    susie_ash_out  <- run_susie_ash(data, precomp, L, K.length = K.length, upper_bound = upper_bound,
                                    intercept = TRUE, standardize = FALSE)
    susie_inf_out  <- run_susie_inf(data, precomp, L, intercept = TRUE, standardize = FALSE)
    fineboost_out  <- run_fineboost(data, null_max, intercept = TRUE, standardize = FALSE)

    # Evaluate metrics from all methods using the evaluation function
    metrics <- evaluate_method_performance(
      susie_out     = susie_out,
      susie_ash_out = susie_ash_out,
      susie_inf_out = susie_inf_out,
      fineboost_out = fineboost_out,
      causal        = data$causal_indices,
      data          = data
    )

    # Save the metrics, data, and fine-mapping outputs
    all_metrics[[i]] <- metrics

    # Remove large objects no longer needed to free memory
    rm(ld_block, precomp, susie_out, susie_ash_out, susie_inf_out, fineboost_out, data)
    gc()
  }

  # Compile all results into a list including simulation parameters
  simulation_results <- list(
    all_metrics = all_metrics,
    parameters  = list(
      num_simulations = num_simulations,
      h2_total        = h2_total,
      K               = K,
      L               = L,
      null_max        = null_max,
      ld_mode         = ld_mode,
      K.length        = K.length,
      upper_bound     = upper_bound,
      LD_blocks_dir   = LD_blocks_dir
    )
  )

  # Save Simulation Results as RDS File
  output_dir <- "/home/apm2217/output"
  file_name <- paste0("numIter", num_simulations,
                      "_h2total", h2_total,
                      "_K", K,
                      "_L", L,
                      "_nullMax", null_max,
                      "_ldMode", ld_mode,
                      "_Klength", K.length,
                      "_upperBound", upper_bound)

  saveRDS(simulation_results, file.path(output_dir, paste0(file_name, ".rds")))

  # Return all results
  return(simulation_results)
}

# Run the simulation and save the results
results <- simulation(
  num_simulations = NULL,
  h2_total        = NULL,
  K               = NULL,
  L               = NULL,
  null_max        = NULL,
  ld_mode         = NULL,
  LD_blocks_dir   = "LD_blocks_precomputed"
)